In [2]:
import os, sys
from numpy import mat
from pymatgen.ext.matproj import MPRester
import json

In [3]:
print(f"Python version info: {sys.version}")

Python version info: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [4]:
API_TOKEN = os.environ.get("MPTOKEN")
# print(API_TOKEN)

## Helper Functions

In [5]:
def pprint(obj: object):
    print(json.dumps(obj, indent=2))
    
def save_findings(obj: object):
    with open("../TMR_Data.json", 'w') as fo:
        json.dump(obj, fo, indent=2)

## Actial data cleaning and aquisition

In [6]:
## Get possible ternery metal nitrats from the materials project.
# Search critea 
#   - All materials with nitrogen in its formula 
#   - these materials must have three elements
#   - then these materials must have a band gap value of less than 1 (logically a metal)

mpr = MPRester(API_TOKEN)

crt = {
    'elements': {'$all': ['N']},
    'nelements': 3,
    # 'band_gap': {'$lt': 1},
    # 'anonymous_formula': {"A": 1, "B": 1, "C": 3}
}
prt = ['material_id', 'pretty_formula', 'full_formula', 'band_gap', 'unit_cell_formula', 'elements']

materials = mpr.query(criteria=crt, properties=prt)

100%|██████████| 4529/4529 [00:15<00:00, 283.11it/s]


In [7]:
for i in materials[:5]: 
    print(i)

print("#"*50, f"\n\nFound {len(materials)} possible candidates")

{'material_id': 'mp-1097065', 'pretty_formula': 'Li2HfN2', 'full_formula': 'Li2Hf1N2', 'band_gap': 1.9339000000000004, 'unit_cell_formula': {'Li': 2.0, 'Hf': 1.0, 'N': 2.0}, 'elements': ['Hf', 'Li', 'N']}
{'material_id': 'mp-1182541', 'pretty_formula': 'B24N4O', 'full_formula': 'B96N16O4', 'band_gap': 0.5880999999999998, 'unit_cell_formula': {'B': 96.0, 'N': 16.0, 'O': 4.0}, 'elements': ['B', 'N', 'O']}
{'material_id': 'mp-1189871', 'pretty_formula': 'Zr5Sn3N', 'full_formula': 'Zr10Sn6N2', 'band_gap': 0.0, 'unit_cell_formula': {'Zr': 10.0, 'Sn': 6.0, 'N': 2.0}, 'elements': ['N', 'Sn', 'Zr']}
{'material_id': 'mp-1192449', 'pretty_formula': 'NClO4', 'full_formula': 'N4Cl4O16', 'band_gap': 0.0, 'unit_cell_formula': {'N': 4.0, 'Cl': 4.0, 'O': 16.0}, 'elements': ['Cl', 'N', 'O']}
{'material_id': 'mp-1196573', 'pretty_formula': 'NaYbN4', 'full_formula': 'Na8Yb8N32', 'band_gap': 0.0, 'unit_cell_formula': {'Na': 8.0, 'Yb': 8.0, 'N': 32.0}, 'elements': ['N', 'Na', 'Yb']}
#######################

Create a new materials object that includes the material type, ie metal or non metal

In [8]:
## Create a list that holds the different element groups
with open("../my_elements_data.json") as fo:
    my_elements_data = json.load(fo)


element_groups = list(set([i['GroupBlock'] for i in my_elements_data]))
print(element_groups)

['Post-transition metal', 'Transition metal', 'Metalloid', 'Nonmetal', 'Alkali metal', 'Actinide', 'Halogen', 'Lanthanide', 'Noble gas', 'Alkaline earth metal']


In [9]:
## Read my data for metals and non metals
with open("../my_elements_data.json") as fo:
    my_elements_data = json.load(fo)
    
# print(my_elements_data[:1])


my_classified_data_list = []
## Create new entry for every material object
for material in materials:
    my_classified_data = {}
    
    my_classified_data.update(material)
    
    # print(f"Processing - {material['pretty_formula']}")
    descriptive = []
    for element in material['elements']:
        # print(element)
        
        for i in my_elements_data:
            if i['Symbol'] == element:
                
                descriptive.append(f"{element} - {i['Name']} - {i['GroupBlock']}")
                # print(f"{element} - {i['Name']} is a {i['GroupBlock']}")
                # print(descriptive)
            
            my_classified_data['descriptive'] = descriptive
    
    my_classified_data_list.append(my_classified_data)
    # print(json.dumps(my_classified_data, indent=2))
    # print("#"*60, '\n\n')


pprint(my_classified_data_list[:2])

[
  {
    "material_id": "mp-1097065",
    "pretty_formula": "Li2HfN2",
    "full_formula": "Li2Hf1N2",
    "band_gap": 1.9339000000000004,
    "unit_cell_formula": {
      "Li": 2.0,
      "Hf": 1.0,
      "N": 2.0
    },
    "elements": [
      "Hf",
      "Li",
      "N"
    ],
    "descriptive": [
      "Hf - Hafnium - Transition metal",
      "Li - Lithium - Alkali metal",
      "N - Nitrogen - Nonmetal"
    ]
  },
  {
    "material_id": "mp-1182541",
    "pretty_formula": "B24N4O",
    "full_formula": "B96N16O4",
    "band_gap": 0.5880999999999998,
    "unit_cell_formula": {
      "B": 96.0,
      "N": 16.0,
      "O": 4.0
    },
    "elements": [
      "B",
      "N",
      "O"
    ],
    "descriptive": [
      "B - Boron - Metalloid",
      "N - Nitrogen - Nonmetal",
      "O - Oxygen - Nonmetal"
    ]
  }
]


Now We check if the material passes our search criteria of Metal-Metal-Nitrogen

In [10]:
# print(json.dumps(my_classified_data_list[:1], indent=2))

final_materials_list = []
## Metal Groups list
metal_groups = ['Transition metal', 'Actinide', 'Lanthanide', 'Alkali metal', 'Post-transition metal', 'Alkaline earth metal']

## Go through all the materials anch check their description for metal groups, metal group, and nitrogen
for material in my_classified_data_list:
    # pprint(material)
    
    # print(material['pretty_formula'], list(material['unit_cell_formula'].keys()), [i.split(' - ') for i in material['descriptive']])
    
    ## Represent the material formula in terma of its group
    element_group_formula = []
    for i in list(material['unit_cell_formula'].keys()):
        for j in [i.split(' - ') for i in material['descriptive']]:
            if j[0] == i:
                if len(element_group_formula) < 2:
                    element_group_formula.append(j[2])
                else:
                    element_group_formula.append(j[1])
    
    ## Check the material group-formula if matches M-M-N
    # print(material['pretty_formula'], element_group_formula)
    if (element_group_formula[0] in metal_groups) and (element_group_formula[1] in metal_groups) and (element_group_formula[2].lower() in "Nitrogen".lower()):
        # print(material['pretty_formula'], element_group_formula)
        final_materials_list.append(material)
    
    
    # break

# final_materials_list = set(final_materials_list)
pprint(final_materials_list[:2])
print(f"Found {len(final_materials_list)} possible candidates (unique ids = {len(set([mat['material_id'] for mat in final_materials_list]))})")
    
        
    

[
  {
    "material_id": "mp-1097065",
    "pretty_formula": "Li2HfN2",
    "full_formula": "Li2Hf1N2",
    "band_gap": 1.9339000000000004,
    "unit_cell_formula": {
      "Li": 2.0,
      "Hf": 1.0,
      "N": 2.0
    },
    "elements": [
      "Hf",
      "Li",
      "N"
    ],
    "descriptive": [
      "Hf - Hafnium - Transition metal",
      "Li - Lithium - Alkali metal",
      "N - Nitrogen - Nonmetal"
    ]
  },
  {
    "material_id": "mp-1189871",
    "pretty_formula": "Zr5Sn3N",
    "full_formula": "Zr10Sn6N2",
    "band_gap": 0.0,
    "unit_cell_formula": {
      "Zr": 10.0,
      "Sn": 6.0,
      "N": 2.0
    },
    "elements": [
      "N",
      "Sn",
      "Zr"
    ],
    "descriptive": [
      "N - Nitrogen - Nonmetal",
      "Sn - Tin - Post-transition metal",
      "Zr - Zirconium - Transition metal"
    ]
  }
]
Found 2236 possible candidates (unique ids = 2236)


In [11]:
## Write findings to disk
final_materials_list_out = {}

my_outro = {}
my_outro['Total-Candidates'] = len(final_materials_list)
# my_outro['Search-Criteria'] = crt
# my_outro['Pulled-Properties'] = prt

final_materials_list_out['summary'] = my_outro
final_materials_list_out['candidates'] = final_materials_list

save_findings(final_materials_list_out)